## External libs in RELAY
Shows how to use external libraries


In [3]:
import tvm
from tvm import te
import numpy as np
from tvm.contrib import graph_runtime as runtime
from tvm import relay
from tvm.relay import testing

Now build a simple network

In [4]:
out_channels = 16
batch_size = 1

data = relay.var("data", relay.TensorType((batch_size, 3, 224, 224), "float32"))
weight = relay.var("weight")
bn_gamma = relay.var("bn_gamma")
bn_beta = relay.var("bn_beta")
bn_mmean = relay.var("bn_mean")
bn_mvar = relay.var("bn_var")

simple_net = relay.nn.conv2d(
    data=data, weight=weight, kernel_size=(3, 3), channels=out_channels, padding=(1, 1)
)
simple_net = relay.nn.batch_norm(simple_net, bn_gamma, bn_beta, bn_mmean, bn_mvar)[0]
simple_net = relay.nn.relu(simple_net)
simple_net = relay.Function(relay.analysis.free_vars(simple_net), simple_net)

data_shape = (batch_size, 3, 224, 224)
net, params = testing.create_workload(simple_net)

Run with CUDA

In [5]:
import logging

logging.basicConfig(level=logging.DEBUG)  # to dump TVM IR after fusion

target = "cuda"
lib = relay.build_module.build(net, target, params=params)

ctx = tvm.context(target, 0)
data = np.random.uniform(-1, 1, size=data_shape).astype("float32")
module = runtime.GraphModule(lib["default"](ctx))
module.set_input("data", data)
module.run()
out_shape = (batch_size, out_channels, 224, 224)
out = module.get_output(0, tvm.nd.empty(out_shape))
out_cuda = out.asnumpy()

INFO:autotvm:Download pre-tuned parameters package from https://raw.githubusercontent.com/uwsampl/tvm-distro/master/tophub/cuda_v0.09.log
...100%, 0.40 MB, 388 KB/s, 1 seconds passed
DEBUG:autotvm:Finish loading 688 records
INFO:compile_engine:Using injective.cpu for add based on highest priority (10)
INFO:compile_engine:Using injective.cpu for sqrt based on highest priority (10)
INFO:compile_engine:Using injective.cpu for divide based on highest priority (10)
INFO:compile_engine:Using injective.cpu for multiply based on highest priority (10)
INFO:compile_engine:Using injective.cpu for expand_dims based on highest priority (10)
INFO:compile_engine:Using injective.cpu for negative based on highest priority (10)
INFO:compile_engine:Using injective.cpu for add based on highest priority (10)
INFO:compile_engine:Using conv2d_nchw.cuda for nn.conv2d based on highest priority (10)
INFO:compile_engine:Using injective.cuda for multiply based on highest priority (10)
INFO:compile_engine:Using in

TVMError: Traceback (most recent call last):
  [bt] (8) 9   libtvm.dylib                        0x000000011c572a28 TVMFuncCall + 72
  [bt] (7) 8   libtvm.dylib                        0x000000011c3e3a0e std::__1::__function::__func<tvm::relay::backend::RelayBuildModule::GetFunction(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&, tvm::runtime::ObjectPtr<tvm::runtime::Object> const&)::'lambda1'(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*), std::__1::allocator<tvm::relay::backend::RelayBuildModule::GetFunction(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&, tvm::runtime::ObjectPtr<tvm::runtime::Object> const&)::'lambda1'(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)>, void (tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)>::operator()(tvm::runtime::TVMArgs&&, tvm::runtime::TVMRetValue*&&) + 62
  [bt] (6) 7   libtvm.dylib                        0x000000011c3e3ba6 tvm::relay::backend::Rel